# Prototype Spread and Transition with ILM Approach

In [8]:
using CovidSim

In [9]:
using StatsBase
using DelimitedFiles
using Distributions
using YAML

In [10]:
ilmat = readdlm("../data/ilmtestdata.csv", ',',Int, header=true)[1]
ilmat = repeat(ilmat, 10_000)
refresh = copy(ilmat)

160000×4 Array{Int64,2}:
 1  2  0   0
 1  3  0   0
 1  4  0   0
 1  5  0   0
 2  1  5   2
 2  2  5   9
 2  3  6   2
 2  4  6   9
 2  5  7   2
 2  1  7   9
 2  2  8   2
 2  3  8   9
 3  4  6   9
 ⋮        
 2  1  5   2
 2  2  5   9
 2  3  6   2
 2  4  6   9
 2  5  7   2
 2  1  7   9
 2  2  8   2
 2  3  8   9
 3  4  6   9
 3  5  6  15
 4  3  8  15
 4  4  8  19

## Spread

In [11]:
spfilename="../parameters/spread_params.yml"
spread_params = CovidSim.read_spread_params(spfilename)
contact_factors = spread_params[:contact_factors]
touch_factors = spread_params[:touch_factors]
send_risk = spread_params[:send_risk]
recv_risk = spread_params[:recv_risk]
riskmx = CovidSim.send_risk_by_recv_risk(send_risk, recv_risk) # (lags, agegrp);

In [12]:
filt_spread_bit = ilmat[:,1] .== 2
filt_spread_idx = findall(filt_spread_bit)
@show size(filt_spread_idx,1) == count(filt_spread_bit)
size(filt_spread_idx,1)

size(filt_spread_idx, 1) == count(filt_spread_bit) = true


80000

In [15]:
# for this test case lets have fewer people who are infected
# resetidx = sample(findall(spreadidx),72000, replace=false)
# ilmat[resetidx,1] .= 1
# index of all spreaders
filt_spread_idx = findall(ilmat[:, cpop_status] .== 2)
n_spreaders = size(filt_spread_idx, 1);

In [26]:
# no of contacts for each spreader
time_it = @elapsed begin
    n_spreaders = size(filt_spread_idx, 1);
#     @show n_spreaders

    contacts_per_spreader = zeros(Int, size(filt_spread_idx,1),2) # second column for lag of the spreader
    for i in 1:size(contacts_per_spreader, 1)  # cond, agegrp
        scale = contact_factors[ilmat[filt_spread_idx[i], cpop_cond]-4, ilmat[filt_spread_idx[i], cpop_agegrp]]
        contacts_per_spreader[i, 1] = round(Int,rand(Gamma(1.3,scale))) # assume density_factor = 1.0
        contacts_per_spreader[i, 2] = ilmat[filt_spread_idx[i], cpop_lag]   # lag of the spreader who made this contact
    end
    n_contacts = sum(contacts_per_spreader[:,1])
#     @show n_contacts

    # assign the contacts 
    filt_contactable_idx = findall(ilmat[:, cpop_status] .!= dead)
    n_contactable = size(filt_contactable_idx, 1)
#     @show n_contactable
    choose_contacts = sample(filt_contactable_idx, min(n_contacts, n_contactable), replace=false)
    n_contacts = size(choose_contacts,1)
#     @show n_contacts, "2nd time"

    # determine contacts are consequential touches and if newly infected
    n_touched = 0
    n_newly_infected = 0

    contact_ptr = 0
    for spr in 1:n_spreaders
        nc = contacts_per_spreader[spr,1]
        lag_spr = contacts_per_spreader[spr, 2]
        # who are the contacts for this spreader?
            contact_selector = (contact_ptr+1):(contact_ptr+nc)
            contact_ptr += nc
        contact_persons = choose_contacts[contact_selector]
#         spr > 50 && break
        for contact_person in contact_persons
#             @show spr,contact_person

            status = ilmat[contact_person, cpop_status]
            characteristic =  status in [1,3] ? [1,0,2][status] : ilmat[contact_person, cpop_cond]-2 # max(0,ilmat[person, cpop_cond]-2
            @debug characteristic < 1 && error("bad characteristic value")
            agegrp = ilmat[contact_person, cpop_agegrp]
            touched = rand(Binomial(1, touch_factors[characteristic, agegrp]))
            n_touched += touched
            if touched == 1 && characteristic == unexposed
                prob = riskmx[lag_spr, agegrp]
                newly_infected = rand(Binomial(1, prob))
                if newly_infected == 1
                    ilmat[contact_person, cpop_cond] = nil # nil === asymptomatic or pre-symptomatic
                    ilmat[contact_person, cpop_status] = infectious
                end
                n_newly_infected += newly_infected
            end
        end
    end
    
end

@show time_it
@show n_touched
@show n_newly_infected

time_it = 0.133434379
n_touched = 45249
n_newly_infected = 4174


4174

In [17]:
spreadidx = findall(ilmat[:, cpop_status] .== 2)
size(spreadidx,1)

83700

In [18]:
ilmat

160000×4 Array{Int64,2}:
 1  2  0   0
 1  3  0   0
 1  4  0   0
 1  5  0   0
 2  1  5   2
 2  2  5   9
 2  3  6   2
 2  4  6   9
 2  5  7   2
 2  1  7   9
 2  2  8   2
 2  3  8   9
 3  4  6   9
 ⋮        
 2  1  5   2
 2  2  5   9
 2  3  6   2
 2  4  6   9
 2  5  7   2
 2  1  7   9
 2  2  8   2
 2  3  8   9
 3  4  6   9
 3  5  6  15
 4  3  8  15
 4  4  8  19

## Transition

In [19]:
std_file = "../parameters/dec_tree_all_25.yml"
treedict, decpoints = setup_dt(std_file)
treedict

Dict{Any,Any} with 5 entries:
  4 => Dict{Any,Any}([9, 5]=>Dict{String,Array{T,1} where T}("probs"=>[0.62, 0.…
  2 => Dict{Any,Any}([9, 5]=>Dict{String,Array{T,1} where T}("probs"=>[0.85, 0.…
  3 => Dict{Any,Any}([9, 5]=>Dict{String,Array{T,1} where T}("probs"=>[0.9, 0.1…
  5 => Dict{Any,Any}([9, 5]=>Dict{String,Array{T,1} where T}("probs"=>[0.5, 0.5…
  1 => Dict{Any,Any}([9, 5]=>Dict{String,Array{T,1} where T}("probs"=>[0.9, 0.1…

In [20]:
treedict[1][[25,7]]

Dict{String,Array{T,1} where T} with 3 entries:
  "probs"    => [0.976, 0.024]
  "branches" => Dict{Any,Any}[Dict("tocond"=>3,"next"=>[0, 0],"pr"=>0.976), Dic…
  "outcomes" => [3, 4]

In [21]:
function precalc_agegrp_filt(dat)
    agegrp_filt_bit = Dict(agegrp => dat[:, cpop_agegrp] .== agegrp for agegrp in agegrps)
    agegrp_filt_idx = Dict(agegrp => findall(agegrp_filt_bit[agegrp]) for agegrp in agegrps)
    return agegrp_filt_bit, agegrp_filt_idx
end
agegrp_filt_bit, agegrp_filt_idx = precalc_agegrp_filt(ilmat);

In [22]:
function transition!(dt, all_decpoints, locale, dat)  

    # @assert (length(locale) == 1 || typeof(locale) <: NamedTuple) "locale must be a single integer or NamedTuple"
    
    for agegrp in agegrps
        tree = dt[agegrp]
        for node in keys(tree)
            nodelag, fromcond = node
            for branch in tree[node]["branches"]
                filt = ( (dat[agegrp_filt_idx[agegrp],cpop_cond] .== fromcond) .& 
                         (dat[agegrp_filt_idx[agegrp],cpop_status] .== infectious) .&
                         (dat[agegrp_filt_idx[agegrp], cpop_lag] .== nodelag)  )
                
                    # boolean filt = ( (dat[:,cpop_cond] .== fromcond) .& 
                    #                  (dat[:,cpop_status] .== infectious) .&
                    #                  (dat[:, cpop_lag] .== nodelag) .&
                    #                  (agegrp_filt_bit[agegrp])  )

                filt = agegrp_filt_idx[agegrp][filt]
                cnt_transition = size(filt, 1)  
                
                if cnt_transition > 0
                    
                    probs = tree[node]["probs"]
                    outcomes = tree[node]["outcomes"]
                    
                    choices = rand(Categorical(probs), cnt_transition) 
                                        
                    for (idx, person) in enumerate(filt)   # findall(filt)                       
                        new_stat_cond = outcomes[choices[idx]]
                        if new_stat_cond in (dead, recovered)  # change status
                            dat[person, cpop_status] = new_stat_cond
                        else   # change disease condition
                            dat[person, cpop_cond] = new_stat_cond
                        end  # if/else
                    end  # for (idx, person)
                    
                end  # if cnt_transition
            end  # for branch
        end  # for node
    end  #for agegrp

    # bump everyone who is still infectious all at once in one go
    filt = (dat[:,cpop_status] .== infectious)
    dat[filt, cpop_lag] .+= 1   
end  
    

transition!(treedict, decpoints, 38015, ilmat);   
ilmat


160000×4 Array{Int64,2}:
 1  2  0   0
 1  3  0   0
 1  4  0   0
 1  5  0   0
 2  1  5   3
 3  2  5   9
 2  3  6   3
 2  4  6  10
 2  5  7   3
 2  1  7  10
 2  2  8   3
 2  3  8  10
 3  4  6   9
 ⋮        
 2  1  5   3
 3  2  5   9
 2  3  6   3
 2  4  6  10
 2  5  7   3
 2  1  7  10
 2  2  8   3
 2  3  8  10
 3  4  6   9
 3  5  6  15
 4  3  8  15
 4  4  8  19

In [23]:
count((ilmat - refresh) .!= 0)

93592

# Seed

In [24]:
CovidSim.make_sick!(ilmat; cnt=[3,3], fromage=[2,3], tocond=nil)

#=
function seed!(day, cnt, lag, cond, agegrps, dat)  #  locale,
    if day == ctr[:day] ]
        
        make_sick!(dat; cnt, fromage, tocond, tolag=1)
        
    end 
end
=#

In [25]:
count((ilmat - refresh) .!= 0)

93610